In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
data=pd.read_csv('muse_v3.csv')

In [ ]:
data.columns

In [ ]:
data=data.drop(columns=['lastfm_url','artist','number_of_emotion_tags','valence_tags','arousal_tags','dominance_tags','mbid','spotify_id','genre'])
data

In [ ]:
x=data['track'].values
x

In [ ]:
y=data['seeds'].values
y

In [ ]:
sentiments=y
emotions=[]
for element in sentiments:
  start_index=element.find("'")+1
  end_index=min(element.find(']')-1,element.find(',')-1)
  emotions.append(element[start_index:end_index])
count=0
unique_emotions=[]
for element in emotions:
  if not element in unique_emotions:
    unique_emotions.append(element)

print(unique_emotions)

In [ ]:
data=data.rename(columns={'seeds': 'emotions'})
data

In [ ]:
data['emotions']=emotions
data

In [ ]:
emotion_number_directory={}
id=0
for element in unique_emotions:
  emotion_number_directory[element]=id
  id+=1
emotion_number_directory

In [ ]:
pip install gensim

In [ ]:
pip install nltk

In [ ]:
import re

tracks=data['track'].values
processed=[]
for track in tracks:
  processed.append(track.lower())
for i in range(len(processed)):
  processed[i]=re.sub('[^a-zA-Z]',' ',processed[i])
for i in range(len(processed)):
  processed[i]=re.sub(r'\s+',' ',processed[i])
processed

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
all_words=[nltk.word_tokenize(title) for title in processed]
all_words

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
for i in range(len(all_words)):
  all_words[i]=[w for w in all_words[i] if w not in stopwords.words('english')]
all_words

In [ ]:
real_words=[]
real_emotions=[]
for i in range(len(all_words)):
  if len(all_words[i])!=0:
    real_words.append(all_words[i])
    real_emotions.append(emotions[i])
all_words=real_words
emotions=real_emotions

x_train,x_test,y_train,y_test=train_test_split(all_words,emotions)

In [ ]:
 x_train

In [ ]:
y_train

In [ ]:
from gensim.models import FastText
model = FastText(vector_size=10, window=3, min_count=1)
model.build_vocab(corpus_iterable=x_train)
model.train(corpus_iterable=x_train,total_examples=len(x_train),epochs=150)


In [ ]:
model.wv['mf']

In [ ]:
model.wv.most_similar(negative=['smart'])

In [ ]:
vocab_length=len(model.wv)
Embedding_dimensions=10

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(x_train)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_length = 60
x_train=pad_sequences(tokenizer.texts_to_sequences(x_train),maxlen=input_length)
x_test=pad_sequences(tokenizer.texts_to_sequences(x_test),maxlen=input_length)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
embedding_matrix = np.zeros((vocab_length+2, Embedding_dimensions))
for word, token in tokenizer.word_index.items():
  if model.wv.__contains__(word):
    embedding_matrix[token]=model.wv.__getitem__(word)
embedding_matrix.shape

In [ ]:
embedding_matrix

In [ ]:
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding

from tensorflow.keras.models import Sequential

In [ ]:
neural_net=Sequential()
neural_net.add(Embedding(input_dim=vocab_length+2,output_dim=Embedding_dimensions,weights=[embedding_matrix],input_length=input_length))
neural_net.add(Bidirectional(LSTM(100,dropout=0.3,return_sequences=True)))
neural_net.add(Bidirectional(LSTM(100,dropout=0.3,return_sequences=True)))
neural_net.add(Conv1D(100, 5, activation='relu'))
neural_net.add(GlobalMaxPool1D())
neural_net.add(Dense(16, activation='relu'))
neural_net.add(Dense(273, activation='softmax'))

In [ ]:
neural_net.summary()

In [ ]:
neural_net.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
parsed_y_train=[]
for label in y_train:
  one_hot_encoded=[]
  for i in range(273):
    if i==emotion_number_directory[label]:
      one_hot_encoded.append(1)
    else:
      one_hot_encoded.append(0)
  parsed_y_train.append(one_hot_encoded)
parsed_y_train[0][133]

In [ ]:
parsed_y_test=[]
for label in y_test:
  one_hot_encoded=[]
  for i in range(273):
    if i==emotion_number_directory[label]:
      one_hot_encoded.append(1)
    else:
      one_hot_encoded.append(0)
  parsed_y_test.append(one_hot_encoded)

In [ ]:
x_train=x_train.tolist()
history=neural_net.fit(x_train,parsed_y_train,batch_size=32,epochs=3,verbose=True)

In [ ]:
history.history

In [ ]:
import pickle

In [ ]:
import joblib

In [ ]:
joblib.dump(neural_net,'nlpmodel.pkl')

In [ ]:
y_pred=neural_net.predict(x_test)

In [ ]:
y_pred

In [ ]:
y_train

In [ ]:
import tensorflow as tf
predictions=[]
for arr in y_pred:
  prediction=tf.argmax(arr)
  datapoint=[]
  for i in range(273):
    if i==prediction:
      datapoint.append(1)
    else:
      datapoint.append(0)
  predictions.append(datapoint)
predictions

In [ ]:
print(predictions[0])

In [ ]:
total_count=0
correct_count=0
for i in range(len(predictions)):
  total_count+=1
  match=True
  for j in range(len(predictions[0])):
    if predictions[i][j]!=parsed_y_test[i][j]:
      match=False
      break
  if match:
    correct_count+=1
print(correct_count)
print(total_count)


In [ ]:
 def prediction_pipeline(title,tokenizer,emotion_number_directory,neural_net):
  parsed_title=title.lower()
  parsed_title=re.sub('[^a-zA-Z]',' ',parsed_title)
  parsed_title=re.sub(r'\s+',' ',parsed_title)
  words=nltk.word_tokenize(parsed_title)
  words=[w for w in words if w not in stopwords.words('english')]
  sample=pad_sequences(tokenizer.texts_to_sequences([words]),maxlen=60)
  prediction_array=neural_net.predict(sample)[0]
  prediction_array=prediction_array.tolist()
  prediction_array=[(prediction_array[i],i) for i in range(273)]
  prediction_array.sort()
  prediction_array.reverse()
  prediction_array=prediction_array[:5]
  for i in range(5):
    target=prediction_array[i][1]
    for key in emotion_number_directory.keys():
      if emotion_number_directory[key]==target:
        prediction_array[i]=(prediction_array[i][0],key)
        break
  return prediction_array

In [ ]:
prediction_pipeline('great day',tokenizer,emotion_number_directory,neural_net)


In [ ]:
joblib.dump(prediction_pipeline,'pipeline.pkl')

In [ ]:
joblib.dump(tokenizer,'tokenizer.pkl')

In [ ]:
joblib.dump(emotion_number_directory,'emotion_number_directory.pkl')